In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
dataset = pd.read_csv('./ml-100k/u.data',sep='\t',header=None)
dataset = dataset.rename(columns={0:'userId',1:'movieId',2:'rating',3:'timestamp'})
dataset

,userId,movieId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [3]:
dataset = dataset.sort_values(by='userId')

In [4]:
dataset

,userId,movieId,rating,timestamp
66567,1,55,5,875072688
62820,1,203,4,878542231
10207,1,183,5,875072262
9971,1,150,5,876892196
22496,1,68,4,875072688
...,...,...,...,...
96823,943,427,4,888639147
70902,943,12,5,888639093
84518,943,284,2,875502192
72321,943,62,3,888640003


In [5]:
#dataset.to_csv('data.csv',encoding='utf-8',index=False)

In [6]:
users = pd.read_csv('./ml-100k/u.user',sep='|',header=None)
users = users.rename(columns={0:'userId',1:'age',2:'sex',3:'occupation',4:'zip_code'})
users

,userId,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


In [39]:
data = pd.read_csv('./ml-100k/u.item',sep='|',header=None) 
movies = data.iloc[:,0:3]
movies = movies.rename(columns={0:'movieId',1:'movieTitle',2:'releaseDate'})

In [40]:
movies

,movieId,movieTitle,releaseDate
0,1,Toy Story (1995),01-Jan-1995
1,2,GoldenEye (1995),01-Jan-1995
2,3,Four Rooms (1995),01-Jan-1995
3,4,Get Shorty (1995),01-Jan-1995
4,5,Copycat (1995),01-Jan-1995
...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998
1678,1679,B. Monkey (1998),06-Feb-1998
1679,1680,Sliding Doors (1998),01-Jan-1998
1680,1681,You So Crazy (1994),01-Jan-1994


In [9]:
display(dataset.info())
display(dataset.describe())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 66567 to 70713
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   userId     100000 non-null  int64
 1   movieId    100000 non-null  int64
 2   rating     100000 non-null  int64
 3   timestamp  100000 non-null  int64
dtypes: int64(4)
memory usage: 3.8 MB


None

,userId,movieId,rating,timestamp
count,100000.00000,100000.000000,100000.000000,1.000000e+05
mean,462.48475,425.530130,3.529860,8.835289e+08
std,266.61442,330.798356,1.125674,5.343856e+06
min,1.00000,1.000000,1.000000,8.747247e+08
25%,254.00000,175.000000,3.000000,8.794487e+08
50%,447.00000,322.000000,4.000000,8.828269e+08
75%,682.00000,631.000000,4.000000,8.882600e+08
max,943.00000,1682.000000,5.000000,8.932866e+08


In [10]:
display(users.info())
display(users.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   userId      943 non-null    int64 
 1   age         943 non-null    int64 
 2   sex         943 non-null    object
 3   occupation  943 non-null    object
 4   zip_code    943 non-null    object
dtypes: int64(2), object(3)
memory usage: 37.0+ KB


None

,userId,age
count,943.000000,943.000000
mean,472.000000,34.051962
std,272.364951,12.192740
min,1.000000,7.000000
25%,236.500000,25.000000
50%,472.000000,31.000000
75%,707.500000,43.000000
max,943.000000,73.000000


In [11]:
display(movies.info())
display(movies.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movieId      1682 non-null   int64 
 1   movieTitle   1682 non-null   object
 2   releaseDate  1681 non-null   object
dtypes: int64(1), object(2)
memory usage: 39.5+ KB


None

,movieId
count,1682.000000
mean,841.500000
std,485.695893
min,1.000000
25%,421.250000
50%,841.500000
75%,1261.750000
max,1682.000000


## 构建模型1

# pyspark


## 构建模型pyspark


In [113]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql import SparkSession
from datetime import *
from pyspark.sql.types import StructField,StructType,IntegerType
from pyspark.sql.functions import rand

In [114]:
#获取现有的SparkSession
spark = SparkSession.builder.getOrCreate()

In [115]:

#指定构建模式
schema = StructType(
[
    StructField('userId',IntegerType(),True),
    StructField('movieId',IntegerType(),True),
    StructField('rating',IntegerType(),True),
    StructField('timestamp',IntegerType(),True)
    
])
#读入数据集
#df = spark.read.csv("ml-100k///u.data",header = False,schema = schema,encoding = 'utf-8',sep='\t')
df = spark.read.csv("data.csv",header = False,schema = schema,encoding = 'utf-8',sep=',')

In [258]:
df.isEmpty()

False

In [116]:
df.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|     55|     5|875072688|
|     1|    203|     4|878542231|
|     1|    183|     5|875072262|
|     1|    150|     5|876892196|
|     1|     68|     4|875072688|
|     1|    201|     3|878542960|
|     1|    157|     4|876892918|
|     1|    184|     4|875072956|
|     1|    210|     4|878542909|
|     1|    163|     4|875072442|
|     1|    271|     2|887431672|
|     1|    146|     4|875071561|
|     1|    176|     5|876892468|
|     1|    144|     4|875073180|
|     1|     53|     3|876893206|
|     1|    160|     4|875072547|
|     1|     33|     4|878542699|
|     1|     44|     5|878543541|
|     1|     97|     3|875073128|
|     1|     14|     5|874965706|
+------+-------+------+---------+
only showing top 20 rows



In [117]:
df.describe().show()

+-------+-----------------+-----------------+------------------+-----------------+
|summary|           userId|          movieId|            rating|        timestamp|
+-------+-----------------+-----------------+------------------+-----------------+
|  count|           100000|           100000|            100000|           100000|
|   mean|        462.48475|        425.53013|           3.52986|8.8352885148862E8|
| stddev|266.6144201275064|330.7983563255851|1.1256735991443163| 5343856.18950269|
|    min|                1|                1|                 1|        874724710|
|    max|              943|             1682|                 5|        893286638|
+-------+-----------------+-----------------+------------------+-----------------+



In [118]:
df.dtypes

[('userId', 'int'),
 ('movieId', 'int'),
 ('rating', 'int'),
 ('timestamp', 'int')]

In [119]:
#temp = df.toDF({'_c0':"userId",'_c1':"movieId",'_c2':"rating",'_c3':"timestamp"})
#temp = df.selectExpr("_c0 as userId","_c1 as movieId","_c2 as rating","_c3 as timestamp")
#temp.show()

In [120]:
temp = df.drop('timestamp')
temp.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|     55|     5|
|     1|    203|     4|
|     1|    183|     5|
|     1|    150|     5|
|     1|     68|     4|
|     1|    201|     3|
|     1|    157|     4|
|     1|    184|     4|
|     1|    210|     4|
|     1|    163|     4|
|     1|    271|     2|
|     1|    146|     4|
|     1|    176|     5|
|     1|    144|     4|
|     1|     53|     3|
|     1|    160|     4|
|     1|     33|     4|
|     1|     44|     5|
|     1|     97|     3|
|     1|     14|     5|
+------+-------+------+
only showing top 20 rows



In [121]:
temp.describe().show()

+-------+-----------------+-----------------+------------------+
|summary|           userId|          movieId|            rating|
+-------+-----------------+-----------------+------------------+
|  count|           100000|           100000|            100000|
|   mean|        462.48475|        425.53013|           3.52986|
| stddev|266.6144201275064|330.7983563255851|1.1256735991443163|
|    min|                1|                1|                 1|
|    max|              943|             1682|                 5|
+-------+-----------------+-----------------+------------------+



In [122]:
temp.dtypes

[('userId', 'int'), ('movieId', 'int'), ('rating', 'int')]

In [123]:
temp.show()
type(temp)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|     55|     5|
|     1|    203|     4|
|     1|    183|     5|
|     1|    150|     5|
|     1|     68|     4|
|     1|    201|     3|
|     1|    157|     4|
|     1|    184|     4|
|     1|    210|     4|
|     1|    163|     4|
|     1|    271|     2|
|     1|    146|     4|
|     1|    176|     5|
|     1|    144|     4|
|     1|     53|     3|
|     1|    160|     4|
|     1|     33|     4|
|     1|     44|     5|
|     1|     97|     3|
|     1|     14|     5|
+------+-------+------+
only showing top 20 rows



pyspark.sql.dataframe.DataFrame

In [124]:
temp.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: integer (nullable = true)



In [125]:
#把数据集划分训练集和测试集(80%,20%)
train,test = temp.randomSplit([0.8,0.2])

In [126]:
train.dtypes

[('userId', 'int'), ('movieId', 'int'), ('rating', 'int')]

In [127]:
train.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|     5|
|     1|      2|     3|
|     1|      3|     4|
|     1|      6|     5|
|     1|      7|     4|
|     1|     10|     3|
|     1|     12|     5|
|     1|     13|     5|
|     1|     14|     5|
|     1|     15|     5|
|     1|     16|     5|
|     1|     19|     5|
|     1|     20|     4|
|     1|     22|     4|
|     1|     23|     4|
|     1|     24|     3|
|     1|     26|     3|
|     1|     27|     2|
|     1|     28|     4|
|     1|     29|     1|
+------+-------+------+
only showing top 20 rows



In [128]:
test.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      4|     3|
|     1|      5|     3|
|     1|      8|     1|
|     1|      9|     5|
|     1|     11|     2|
|     1|     17|     3|
|     1|     18|     4|
|     1|     21|     1|
|     1|     25|     4|
|     1|     34|     2|
|     1|     45|     5|
|     1|     46|     4|
|     1|     47|     4|
|     1|     49|     3|
|     1|     56|     4|
|     1|     59|     5|
|     1|     62|     3|
|     1|     66|     4|
|     1|     71|     3|
|     1|     73|     3|
+------+-------+------+
only showing top 20 rows



In [129]:
#使用ALS来建立推荐模型，采用显性反馈
alsExplicit  = ALS(maxIter=5, regParam=0.01, userCol='userId',itemCol='movieId', ratingCol='rating',alpha=1.0,seed=0)
#把推荐模型放在训练数据上训练
modelExplicit = alsExplicit.fit(train)
#使用训练好的推荐模型对测试集进行预测评分，得到预测评分的数据集
predictionsExplicit = modelExplicit.transform(test)
#显性反馈
predictionsExplicit.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|     1|      4|     3|  3.660544|
|     1|      5|     3| 3.2065282|
|     1|      8|     1| 4.0137224|
|     1|      9|     5|  4.450247|
|     1|     11|     2|  4.300669|
|     1|     17|     3| 2.9935288|
|     1|     18|     4| 3.7635968|
|     1|     21|     1|  2.697554|
|     1|     25|     4| 3.5863023|
|     1|     34|     2|0.91532433|
|     1|     45|     5| 5.0524964|
|     1|     46|     4| 4.0717874|
|     1|     47|     4| 3.4950066|
|     1|     49|     3| 3.9482698|
|     1|     56|     4| 5.1433554|
|     1|     59|     5| 4.8182287|
|     1|     62|     3| 3.2599528|
|     1|     66|     4|  2.947281|
|     1|     71|     3|  3.393766|
|     1|     73|     3|  3.053169|
+------+-------+------+----------+
only showing top 20 rows



In [130]:
predictionsExplicit.dtypes

[('userId', 'int'),
 ('movieId', 'int'),
 ('rating', 'int'),
 ('prediction', 'float')]

In [131]:
test.describe().show()

+-------+------------------+------------------+------------------+
|summary|            userId|           movieId|            rating|
+-------+------------------+------------------+------------------+
|  count|             20077|             20077|             20077|
|   mean| 462.0234596802311| 425.2108382726503| 3.533446232006774|
| stddev|266.45759274665414|328.96862036367594|1.1176282876702133|
|    min|                 1|                 1|                 1|
|    max|               943|              1680|                 5|
+-------+------------------+------------------+------------------+



In [91]:
# 单个用户
#single_user = test.filter(test['userId']==62).select(['movieId','userId','rating'])

In [92]:
#single_user.show(5)

In [93]:
#reccomendations = modelExplicit.transform(single_user)

In [94]:
#reccomendations.sort('prediction',ascending=False).show(5)

In [95]:
# 下方代码有问题，np.random.randint(0,15000)

In [96]:
#x = np.random.randint(0,19000)
#test_ = test.take(x)[np.random.randint(0,x-1)]

In [97]:
#test_

In [98]:
#test_ = test_[0:2]

In [99]:
#schema = StructType(
#[
#    StructField('userId',IntegerType(),True),
#    StructField('movieId',IntegerType(),True),
#    #StructField('rating',IntegerType(),True),
#    
#])
#test_demo = spark.createDataFrame([test_],schema = schema)

In [100]:
#test_demo

In [101]:
#result = modelExplicit.transform(test_demo)

In [102]:
#result.show()

## 模型评估

In [132]:
#去除空值
predictionsExplicit = predictionsExplicit.na.drop(how="any",subset=['prediction'])

In [133]:
#采取对模型的均方根误差进行评估
evaluator = RegressionEvaluator().setMetricName("rmse").setLabelCol("rating").setPredictionCol("prediction")

In [134]:
#显性反馈的均方误差
rmseExplicit = evaluator.evaluate(predictionsExplicit)

In [135]:
print("显性反馈的均方误差 = ",rmseExplicit)

显性反馈的均方误差 =  1.0607406237967845


In [ ]:
#去除空值
#predictionsExplicit = predictionsExplicit.na.drop(how="any",subset=['prediction'])
#采取对模型的均方根误差进行评估
#evaluator = RegressionEvaluator().setMetricName("rmse").setLabelCol("rating").setPredictionCol("prediction")
#显性反馈的均方误差
#rmseExplicit = evaluator.evaluate(predictionsExplicit)

## 模型应用

In [136]:
modelExplicit.rank

10

In [137]:
movies.iloc[:,0:2]

,movieId,movieTitle
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


In [138]:
# 1、为每个用户生成前 5 个电影推荐
user = modelExplicit.recommendForAllUsers(5)

In [140]:
#转换为pandas.dataframe
df_1 = user.toPandas()
df_1

,userId,recommendations
0,1,"[(1193, 5.932034015655518), (793, 5.6091308593..."
1,2,"[(854, 6.936951160430908), (1170, 6.8154582977..."
2,3,"[(206, 8.405196189880371), (1286, 8.1976528167..."
3,4,"[(320, 9.206167221069336), (1438, 8.9592189788..."
4,5,"[(793, 8.238310813903809), (1286, 7.5262317657..."
...,...,...
938,939,"[(1631, 8.87443733215332), (1425, 8.6181297302..."
939,940,"[(793, 6.4935503005981445), (965, 6.0729646682..."
940,941,"[(1199, 7.911586284637451), (1131, 7.793392181..."
941,942,"[(1120, 6.606384754180908), (1463, 6.264856338..."


In [141]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   userId           943 non-null    int32 
 1   recommendations  943 non-null    object
dtypes: int32(1), object(1)
memory usage: 11.2+ KB


In [232]:
input_user =  '0'#设置开始循环
while input_user.isdigit():    #循环条件判断
    input_user = input("输入用户ID(要求为数字，输入错误自动退出！):") 
    #判断输入是否为正确的用户编号
    if input_user.isdigit()==False:   
        break
    elif int(input_user)>943:
        break
    elif int(input_user)<0:
        break
    # 提取每个用户的recommendations（包含五部电影）   
    user_recommendations = df_1[df_1['userId']==int(input_user)]['recommendations']
    #格式化遍历输出
    for i in range(5):
        temp = user_recommendations.iloc[0][i]
        #rating_predict = temp[1]
        movieTitle = movies[movies['movieId']==temp[0]]['movieTitle'].iloc[0]
        print(f"对于用户：{input_user}；推荐电影：编号:{temp[0]}，标题:{movieTitle}")
        
    print("==================================================================================")
    

输入用户ID(要求为数字，输入错误自动退出！):12
对于用户：12；推荐电影：编号:1166，标题:Love & Human Remains (1993)
对于用户：12；推荐电影：编号:1131，标题:Safe (1995)
对于用户：12；推荐电影：编号:1389，标题:Mondo (1996)
对于用户：12；推荐电影：编号:1120，标题:I'm Not Rappaport (1996)
对于用户：12；推荐电影：编号:1631，标题:Slingshot, The (1993)
输入用户ID(要求为数字，输入错误自动退出！):3
对于用户：3；推荐电影：编号:206，标题:Akira (1988)
对于用户：3；推荐电影：编号:1286，标题:Shall We Dance? (1937)
对于用户：3；推荐电影：编号:793，标题:Crooklyn (1994)
对于用户：3；推荐电影：编号:703，标题:Widows' Peak (1994)
对于用户：3；推荐电影：编号:1225，标题:Angus (1995)
输入用户ID(要求为数字，输入错误自动退出！):45
对于用户：45；推荐电影：编号:337，标题:House of Yes, The (1997)
对于用户：45；推荐电影：编号:837，标题:Meet John Doe (1941)
对于用户：45；推荐电影：编号:1172，标题:Women, The (1939)
对于用户：45；推荐电影：编号:945，标题:Charade (1963)
对于用户：45；推荐电影：编号:189，标题:Grand Day Out, A (1992)
输入用户ID(要求为数字，输入错误自动退出！):-2


In [143]:
#df.select(alsExplicit.getUserCol()).distinct().orderBy(rand(seed=2)).limit(3).show()

In [144]:
# 2、为一组随机指定的用户生成前 5 个电影推荐
#（也可以自行指定：自行构造userId的dataframe(pyspark.sql)即可）
users = df.select(alsExplicit.getUserCol()).distinct().orderBy(rand(seed=2)).limit(3)
users_recommendations = modelExplicit.recommendForUserSubset(users,5)

In [222]:
#转换为pandas.dataframe
df_2 = users_recommendations.toPandas()

In [219]:
for user in df_2.userId:  #获得用户Id
    #提取每个用户的recommendations（包含五部电影）
    recommendations = df_2[df_2.userId==user]['recommendations'].iloc[0] 
    #格式化遍历输出
    for i in range(5):
        user_recommendation = recommendations[i]
        movieTitle = movies[movies['movieId']==user_recommendation[0]]['movieTitle'].iloc[0]
        print(f"对于用户:{user};推荐电影：编号:{user_recommendation[0]}，标题:{movieTitle}")
    print("==================================================================================")

对于用户:652;推荐电影：编号:1425,标题:I'll Do Anything (1994)
对于用户:652;推荐电影：编号:1097,标题:Hate (Haine, La) (1995)
对于用户:652;推荐电影：编号:725,标题:Exit to Eden (1994)
对于用户:652;推荐电影：编号:1121,标题:Umbrellas of Cherbourg, The (Parapluies de Cherbourg, Les) (1964)
对于用户:652;推荐电影：编号:865,标题:Ice Storm, The (1997)
对于用户:376;推荐电影：编号:1015,标题:Shiloh (1997)
对于用户:376;推荐电影：编号:320,标题:Paradise Lost: The Child Murders at Robin Hood Hills (1996)
对于用户:376;推荐电影：编号:1426,标题:Grace of My Heart (1996)
对于用户:376;推荐电影：编号:745,标题:Ruling Class, The (1972)
对于用户:376;推荐电影：编号:613,标题:My Man Godfrey (1936)
对于用户:46;推荐电影：编号:1256,标题:Designated Mourner, The (1997)
对于用户:46;推荐电影：编号:1131,标题:Safe (1995)
对于用户:46;推荐电影：编号:1128,标题:Heidi Fleiss: Hollywood Madam (1995) 
对于用户:46;推荐电影：编号:1462,标题:Thieves (Voleurs, Les) (1996)
对于用户:46;推荐电影：编号:1411,标题:Barbarella (1968)


In [257]:
# 3、为每部电影生成前 5 个用户推荐
movie_ = modelExplicit.recommendForAllItems(5)
df_3 = movie_.toPandas()

In [256]:
input_movie =  '0'#设置开始循环
while input_movie.isdigit():    #循环条件判断
    input_movie = input("输入电影ID(要求为数字，输入错误自动退出！):")     
    #判断输入是否为正确的电影ID
    if input_movie.isdigit()==False:   
        break
    elif int(input_movie)>1682:
        break
    elif int(input_movie)<0:
        break
        
    # 提取每个电影的recommendations（包含五个用户）   
    movie_recommendations = df_3[df_3['movieId']==int(input_movie)]['recommendations']
    #格式化遍历输出
    movieTitle = movies[movies['movieId']==int(input_movie)]['movieTitle'].iloc[0]
    print(f"对于电影：编号：{input_movie}，标题:{movieTitle}；5个推荐用户：")
    for i in range(5):
        temp = movie_recommendations.iloc[0][i]
        print(f"用户编号:{temp[0]}")       
    print("==================================================================================")   

输入电影ID(要求为数字，输入错误自动退出！):3
对于电影：编号：3，标题:Four Rooms (1995)；5个推荐用户：
用户编号:153
用户编号:220
用户编号:362
用户编号:353
用户编号:34
输入电影ID(要求为数字，输入错误自动退出！):67
对于电影：编号：67，标题:Ace Ventura: Pet Detective (1994)；5个推荐用户：
用户编号:39
用户编号:745
用户编号:789
用户编号:812
用户编号:914
输入电影ID(要求为数字，输入错误自动退出！):1600
对于电影：编号：1600，标题:Guantanamera (1994)；5个推荐用户：
用户编号:180
用户编号:309
用户编号:649
用户编号:888
用户编号:242
输入电影ID(要求为数字，输入错误自动退出！):e
